# ReID Project

In [ ]:
"""
Outline:

a basline for Person ReID
market1501 --> ask @ TA
training data: '00001': [img1, img2, img3,...]
testing data: [img ...]? 

Given a query photo, recognize person ID ?  < == image search / retrieval
Gallery : image database <== training data + testing data (only a few)

q, g_i in G : learn a (similarity / distance) function: s(q, g_i) ? 

Suppose q and g_i are feature vectors, s(q, g_i)  = q dot_product g_i 

ConvNet e.g. ResNet-50 / MobileNet  == > Train a ConvNet

"""

In [1]:
import tensorflow
from tensorflow.keras.optimizers import SGD
img_width = 64
img_height =64
learning_rate = 0.01

optimizer = SGD(learning_rate = learning_rate)
batch_size = 256
nbr_epochs = 20

# define own leraning rate updates
def lr_decay_basic(epoch, initial_lrate):
    decay_epochs = [40, 70]
    if epoch in decay_epochs:
        decay_rate = 0.1
        new_lrate = initial_lrate * decay_rate
        return new_lrate
    else:
        return initial_lrate

data_folder = '/Users/DanDan/Desktop/七月在线/机器学习原理/第六阶段CV推荐NLP实战/CV/Market-1501-v15.09.15/bounding_box_train'
# 12936 images in the training set

In [2]:
# image roots and path, names
import os
data_root = os.path.join(os.getcwd(), data_folder)
image_names = sorted([x for x in os.listdir(data_root) if x.endswith('.jpg')])
# image_name: '0002_c2s1_ooo451_003'

img_name, img_path = zip(*[(img_file_png[:-4], os.path.join(data_root, img_file_png)
                           ) for img_file_png in image_names])

person_id_original_list = [x[:4] for x in img_name]

nbr_person_ids = len(set(person_id_original_list))   # remove duplicate
print('number of person ids', nbr_person_ids)

number of person ids 751


In [3]:
# labelencoder
from sklearn.preprocessing import LabelEncoder
id_encoder = LabelEncoder()
id_encoder.fit(person_id_original_list)
person_id_encoded = id_encoder.transform(person_id_original_list)

In [4]:
from sklearn.model_selection import train_test_split
train_img_path, val_img_path, train_person_ids, val_person_ids = train_test_split(
                                img_path,person_id_encoded, test_size=0.2, random_state= 42)
print('# train images: {}, # val images: {}, # image labels: {}'.format(
                len(train_img_path), len(val_img_path), len(set(train_person_ids))))

# train images: 10348, # val images: 2588, # image labels: 751


In [5]:
# input_shape: (batch, height, width, channels)
# load pretrained MobileNet - backbone
# mobilnetv2 is for smaller datasets
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

cnn_model = MobileNetV2(include_top = False, weights = 'imagenet', alpha=0.5, 
                       input_shape = (img_height, img_width, 3), pooling = 'max')

In [6]:
global_pool = cnn_model.layers[-1].output

In [7]:
from tensorflow.keras.layers import Dense, Activation
# from tensorflow.python.keras.layers import Lambda
# from tensorflow.keras.utils import backend as K
from keras import backend as K
dense_normalized = tensorflow.keras.layers.Lambda(
    lambda x: K.l2_normalize(x, axis=1), name='triplet_loss')(global_pool)

dense = Dense(nbr_person_ids)(global_pool)
softmax_output=Activation('softmax')(dense)

from tensorflow.keras.models import Model
triplet_model = Model(cnn_model.input, [dense_normalized, softmax_output])
# triplet_model.summary()


import tensorflow_addons as tfa
triplet_semi_hard_loss = tfa.losses.TripletSemiHardLoss(margin = 0.3)


# Label Smoothing -- > overfitting (one-hot encoding )
#  [0, 1, 0, 0] --  [0.33, 0.9, 0.33, 0.33]
def cross_entropy_label_smoothing(y_true, y_pred):
    from tensorflow.keras.losses import categorical_crossentropy
    label_smoothing = 0.1
    return categorical_crossentropy(y_true, y_pred, label_smoothing = label_smoothing)


USE_Label_Smoothing = True
if USE_Label_Smoothing:
#     from utils import cross_entropy_label_smoothing
    triplet_model.compile(loss = [triplet_semi_hard_loss, cross_entropy_label_smoothing],
                          optimizer = optimizer,
                          metrics = ['accuracy'])
else:
    triplet_model.compile(loss = [triplet_semi_hard_loss, 'categorical_crossentropy'],
                          optimizer = optimizer,
                          metrics = ['accuracy'])

In [12]:
import numpy as np
import cv2
# data augmentation: https:github.com/aleju/imgaug
# pip install imgaug
import imgaug as ia
from imgaug import augmenters as iaa

# Triplet loss  / deep ranking: solution to image retrieval
# e.g. amazon shopping in amazon
# triplet: (anchor, positive, negative)
# loss / target : in feature space, dis(pos, anchor) + margin < dis(neg, anchor)
# sampling the triplet data( anchor, pos, neg) is really important! 
# In the field of person ReID, how to sample data?
# anchor shares the same ID with pos, different from neg
# Hard Triplet loss (Sampling Hard Data --> hard negative)

# from tensorflow.keras.utils import backend as K
def triplet_loss(y_true, y_pred, alpha = 0.3):
    y_pred = K.l2_normalize(y_pred, axis=1)
    
    batch_num = y_pred.shape.as_list()[-1] / 3
    
    anchor = y_pred[:, 0 : batch_num]
    positive = y_pred[:, batch_num : batch_num * 2]
    negative = y_pred[:, batch_num * 2 : batch_num * 3]
    
    # distance between the anchor and the positve
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    # distance bwteeen the anchor and the negative
    neg_dist = K.sum(K.square(anchor - negative), axis=1)
    
    loss = K.maximum(pos_dist - neg_dist + alpha, 0)  # hinge loss


seq = iaa.Sequential()

def load_img_batch(img_path_list, img_label_list, nbr_classes, img_width, img_height):
    batch_size = len(img_path_list)
    
    X_batch = np.zeros((batch_size, img_height, img_width, 3))
    Y_batch = np.zeros((batch_size, nbr_classes)) # label: one-hot encoding

    for i in range(batch_size):
        img_path = img_path_list[i]
        
        
        img_bgr = cv2.imread(img_path) # img.shape(128, 64, 3)
        if img_bgr.shape != (img_height, img_width, 3):
            img_bgr = cv2.resize(img_bgr, (img_width, img_height))
            
        img = img_bgr[:, :, ::-1]

        X_batch[i] = img
        
        if img_label_list is not None:
            label = img_label_list[i]
            Y_batch[i, label] = 1
            
    if img_label_list is not None:
        return X_batch, Y_batch
    else:
        return X_batch
    
def generator_batch_triplet(img_path_list, img_label_list, nbr_classes, 
                            img_width, img_height, P=16, K=4, 
                            shuffle=False, save_to_dir = None, augment=False): 
    
    # img_path_list : ['/home/data/1.jpg', ....]
    # img_label_list: [7, 23, 4, ...]
    # output: yield (X_batch, y_batch)  or X_batch 
    
    N = len(img_path_list)
    if shuffle:
        from sklearn.utils import shuffle as shuffle_tuple
        img_path_list, img_label_list = shuffle_tuple(img_path_list, img_label_list)
        
        
    dic = {}
    for img_label, img_path in zip(img_label_list, img_path_list):
        dic.setdefault(img_label, []).append(img_path)
        
    person_ids_list = [k for k in dic.keys() if len(dic[k]) >= K]
    
    while True:
        import random
        person_ids_sampled = random.sample(person_ids_list, k= 4)
        img_path_sampled = [random.sample((dic[person_id]),k =4) for person_id in person_ids_sampled]
        
        img_path_sampled_list = [ ]
        [img_path_sampled_list.extend(w) for w in img_path_sampled]
        
        person_ids_sampled_list = [ ]
        tmp_sampled_list = [[w] * K for w in person_ids_sampled]
        [person_ids_sampled_list.extend(w) for w in tmp_sampled_list]
        
        y_batch = np.array(person_ids_sampled_list)
        
        X_batch, Y_batch = load_img_batch(img_path_sampled_list,
                                         person_ids_sampled_list,
                                         nbr_classes, img_width, img_height)
        
        if augment:
            X_batch = X_batch.astype(np.uint8)
            X_batch_aug = seq.augment_images(X_batch)
            X_batch = X_batch_aug
            
        X_batch = X_batch / 255.
        X_batch = (X_batch - np.array([0.485, 0.486, 0.406])) / np.array([0.229, 0.224, 0.225])
        yield(X_batch, [y_batch, Y_batch])

In [13]:
# Data Loading, if large size, use batch
# from utils import generator_batch_triplet

train_generator = generator_batch_triplet(img_path_list = train_img_path, 
                                  img_label_list = train_person_ids,
                                 nbr_classes = nbr_person_ids,
                                 img_width=img_width, img_height= img_height,
                                 P = 16, K=4, shuffle=True,
                                 save_to_dir = False, augment = True)

val_generator = generator_batch_triplet(img_path_list = val_img_path,
                               img_label_list = val_person_ids, nbr_classes=nbr_person_ids,
                               img_width=img_width, img_height= img_height,
                               P=16, K=4, shuffle = False,
                               save_to_dir = False, augment= False)


# from tensorflow.keras.callbacks import Checkpoint
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
# CALLBACK
checkpoint = ModelCheckpoint('./cnn_baseline.h5', monitor='val_accuracy',
                            verbose=1, save_best_only=True)
learning_rate_decay = LearningRateScheduler(lr_decay_basic, verbose=1)

triplet_model.fit(train_generator, 
                   steps_per_epoch=len(train_img_path) // batch_size,
                   validation_data = val_generator,
                  validation_steps = len(val_img_path) // batch_size,
                  batch_size=batch_size,verbose=1,shuffle=True,
                  epochs=nbr_epochs,callbacks=[checkpoint, learning_rate_decay])


Epoch 00001: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/20
40/40 [==============================] - 10s 236ms/step - loss: 8.4180 - triplet_loss_loss: 0.2886 - activation_loss: 8.1294 - triplet_loss_accuracy: 0.0016 - activation_accuracy: 0.0016 - val_loss: 7.8572 - val_triplet_loss_loss: 0.2648 - val_activation_loss: 7.5924 - val_triplet_loss_accuracy: 0.0000e+00 - val_activation_accuracy: 0.0000e+00 - lr: 0.0100

Epoch 00002: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 2/20
40/40 [==============================] - 8s 200ms/step - loss: 7.8293 - triplet_loss_loss: 0.2847 - activation_loss: 7.5446 - triplet_loss_accuracy: 0.0016 - activation_accuracy: 0.0016 - val_loss: 8.6057 - val_triplet_loss_loss: 0.2521 - val_activation_loss: 8.3536 - val_triplet_loss_accuracy: 0.0000e+00 - val_activation_accuracy: 0.0000e+00 - lr: 0.0100

Epoch 00003: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 3/20
4

40/40 [==============================] - 9s 232ms/step - loss: 6.8935 - triplet_loss_loss: 0.2444 - activation_loss: 6.6491 - triplet_loss_accuracy: 0.0016 - activation_accuracy: 0.0031 - val_loss: 8.1459 - val_triplet_loss_loss: 0.2092 - val_activation_loss: 7.9368 - val_triplet_loss_accuracy: 0.0000e+00 - val_activation_accuracy: 0.0000e+00 - lr: 0.0100

Epoch 00013: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 13/20
40/40 [==============================] - 7s 184ms/step - loss: 6.9203 - triplet_loss_loss: 0.2477 - activation_loss: 6.6726 - triplet_loss_accuracy: 0.0047 - activation_accuracy: 0.0063 - val_loss: 7.5935 - val_triplet_loss_loss: 0.2173 - val_activation_loss: 7.3762 - val_triplet_loss_accuracy: 0.0000e+00 - val_activation_accuracy: 0.0125 - lr: 0.0100

Epoch 00014: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 14/20
40/40 [==============================] - 7s 182ms/step - loss: 6.9174 - triplet_loss_loss: 0.2425 

# Model Evaluation

In [14]:
img_width = 64
img_height = 128
USE_Label_Smoothing = True
batch_size = 128
model_path = './cnn_baseline.h5'

In [15]:
import os
query_folder = '/Users/DanDan/Desktop/七月在线/机器学习原理/第六阶段CV推荐NLP实战/CV/Market-1501-v15.09.15/query'
# 3368 images in the query set
query_root = os.path.join(os.getcwd(), query_folder)

query_image_names = sorted([x for x in os.listdir(query_root) if x.endswith('.jpg')])

query_img_name, query_img_path = zip(*[(img_file_png[:-4], os.path.join(query_root))
                                     for img_file_png in query_image_names])



gallery_folder = '/Users/DanDan/Desktop/七月在线/机器学习原理/第六阶段CV推荐NLP实战/CV/Market-1501-v15.09.15/gt_bbox'
# 25259 images in the gallery folder

gallery_root = os.path.join(os.getcwd(),gallery_folder)
gallery_image_names = sorted([x for x in os.listdir(gallery_root) if x.endswith('.jpg')])

# remove the duplicated images from the gallery set
gallery_image_names = [x for x in gallery_image_names if x not in query_image_names]
gallery_img_name, gallery_img_path = zip(*[(img_file_png[:-4], os.path.join(gallery_root)) 
                                      for img_file_png in gallery_image_names])

In [16]:
from tensorflow.keras.models import load_model

if USE_Label_Smoothing:
    
#     from utils import cross_entropy_label_smoothing
    model = load_model(model_path, 
                      custom_objects={'cross_entropy_label_smoothing': cross_entropy_label_smoothing})
else:
    model = load_model(model_path)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 32, 32, 16)   432         ['input_10[0][0]']               
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 32, 32, 16)   64          ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 32, 32, 16)   0           ['bn_Conv1[0][0]']               
                                                                                            

In [17]:
dense_features = model.get_layer('global_max_pooling2d_9').output
# 1280-D

from tensorflow.keras.models import Model
model_extract_features = Model(model.input, dense_features)

from tensorflow.keras.optimizers import SGD
optimizer = SGD(learning_rate = 0.01)

model_extract_features.compile(loss = 'categorical_crossentropy',
                              optimizer = optimizer,
                              metrics =['accuracy'])

In [23]:
from tensorflow.keras.utils import generator_batch_predict

ImportError: cannot import name 'generator_batch_predict' from 'tensorflow.keras.utils' (/opt/anaconda3/lib/python3.8/site-packages/keras/api/_v2/keras/utils/__init__.py)

In [18]:
from tensorflow.keras.utils import generator_batch_predict
query_generator = generator_batch_predict(
    img_path_list = query_img_path, img_width = img_width,
                                         img_height = img_height,
                                         batch_size = batch_size)

query_features = model_extract_features.predict(query_generator, verbose=1,
      steps=len(query_img_path) / batch_size + 1)

from sklearn.preprocessing import normalize
query_features = normalize(query_features, norm = 'l2')
# shape: (3368, 1280)
print('query features shape:', query_features.shape)

ImportError: cannot import name 'generator_batch_predict' from 'tensorflow.keras.utils' (/opt/anaconda3/lib/python3.8/site-packages/keras/api/_v2/keras/utils/__init__.py)

In [ ]:
gallery_generator = generator_batch_predict(img_path_list = gallery_img_path,
                                           img_width = img_width,
                                           img_height=img_height,
                                           batch_size=batch_size)

gallery_features = model_extract_features.predict(gallery_generator, verbose=1,
                                    steps=len(gallery_img_path) / batch_size + 1)

gallery_features = normalize(gallery_features, norm ='l2')
print('gallery features shape:', gallery_features.shape)
# shape : (25259, 1280)

In [ ]:
# a -- vector, b --vector, similarity ? Sim(a, b) = dot_product(a, b)
# a -- vector, B -- matrix, similarity ? 
# A -- matrix, B -- matrix, 

import numpy as np
similarity_matrix = np.dot(query_features, np.transpose(gallery_features))
# 3368 X 1280, (25259 X 1280).T

distance_matrix = 1 - similarity_matrix

In [ ]:
idx_list = np.argsort(similarity_matrix, axis=1)
# each row in idx_list is a prediction
# idx_list[row][0] is the index of the prediciton ID

top_1_acc = 0
for i, query_name in enumerate(query_img_name):
    query_id = int(query_name[:4])
    
    pred = model_extract_features.predict(idx_list, verbose=1,
      steps=len(query_img_path) / batch_size + 1)
    
    top_1_acc += pred
    
top_1_acc / len(query_img_name)

In [ ]:
print(tensorflow.__version__)